In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [3]:
from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

Loading csv files

In [11]:
#dataset_TIST2015_Checkins: User ID, Venue ID, UTC time, Time zone
#dataset_TIST2015_POIs: Venue ID, Latitude, Longitude, Venue category name, Country code 
#dataset_TIST2015_Cities: Cityname, Latitude, Longitude, Vanue Category, Country code, Country name, City type
#dataset_UbiComp2016_UserProfile_{CITY}: User ID, Gender, Twitter friend count, Twitter follower count

#Scenario 1: Checkins og POI locations

checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkin = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

city_cols = ['city_name', 'citycenter_latitude', 'citycenter_longitude', 'country_code', 'country_name', 'city_type']
cities = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Cities.csv', sep=',', names=city_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')


#Scenario 2: Users og Checkins

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'gender', 'sex']
ratings_cols = ['user_id', 'venue_id', 'latitude', 'longitude', 'category' 'unix_timestamp']

#Scenario3: Users, Checkins og POI Locations
checkin_cols = ['poi_id', 'gender', 'sex']
venue_cols = ['venue_id', 'user_id', 'latitude', 'longitude', 'category' 'unix_timestamp']
user_cols = ['user_id', 'gender']




C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
print(cities)

        city_name citycenter_latitude citycenter_longitude  country_code  \
0       City_name            latitude            longitude  country_code   
1          Cuiaba             -15.615           -56.093004            BR   
2        Brasilia          -15.792111           -47.897748            BR   
3         Goiania          -16.727004           -49.255001            BR   
4    Campo Grande          -20.450997           -54.615996            BR   
..            ...                 ...                  ...           ...   
411      Brisbane          -27.453996           153.026002            AU   
412        Sydney           -33.88977             151.0282            AU   
413      Adelaide          -34.811676           138.520828            AU   
414     Melbourne          -37.852956           145.075105            AU   
415      Auckland          -36.903996           174.761002            NZ   

     country_name                        city_type  
0    country_name                 

In [ ]:
dim1 = cities[['city_name', 'citycenter_longitude', 'citycenter_latitude', 'city_type', 'country_code']].drop(labels=[0], axis=0)

In [16]:
dim2 = checkin[['user_id', 'poi_id', 'timestamp']].drop(labels=[0], axis=0)

In [23]:
dim3 = pois[['poi_id', 'latitude', 'longitude', 'category', 'country_code']].drop(labels=[0], axis=0)

In [25]:
dataset = dim2.merge(dim3, on='poi_id')#.merge(dim1, on='country_code')

In [ ]:
# Merge Users and Checkins. 
# However, our data does not allow us to merge users and checkin, because we only have data for users that have checked in NYC and Tokyo
#movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

In [27]:
#Split into train and test
train_, test = train_test_split(dataset, test_size=0.1)

In [31]:
ROW_COUNT = train_.shape[0]

In [ ]:
EMBEDDING_SIZE = 10
NUM_USERS = dataset['user_id'].nunique()
NUM_POI = dataset['poi_id'].nunique()

In [29]:
dataset.head()

,user_id,poi_id,timestamp,latitude,longitude,category,country_code
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,55.696132,37.557842,Comedy Club,RU
1,190571,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
2,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 05:49:28 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
3,227763,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:10:31 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
4,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:53:32 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR


In [ ]:
UNIQUE_MOVIE_IDS = dataset['movie_id'].unique()

Define the model

In [92]:
class EmbModel(Model):
    def __init__(self):
        super(EmbModel, self).__init__()
        self.model = self.init_model()
        
    def call(self, inputs):
        return
    
    def init_model(self):
        poi_input = keras.Input(shape=(1,), name='poi_id')
        poi_emb = layers.Embedding(output_dim=10, input_dim=1, input_length=1, name='poi_emb')(poi_input)
    
        user_input = keras.Input(shape=(1,), name='user_id')
        user_emb = layers.Embedding(output_dim=10, input_dim=1, input_length=1, name='user_emb')(user_input)
        print(user_emb.shape)
    
        dot = layers.Dot(axes=(1))([poi_emb, user_emb])
        model = Model([poi_input, user_input], dot)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        if len(data) == 3:
            real_data, labels, sample_weight = data
        else:
            sample_weight = None
            real_images, labels = data
        
        batch_size = real_images.get_shape()[0]
        
        for i in range(self.d_steps):
            
            random_latent_vectors = tf.convert_to_tensor(tf.random.normal(shape=(batch_size, self.latent_dim)))
            
            with tf.GradientTape() as tape:
                dotproduct = self.model([real_data[0], real_data[1]])
                
                # Loss function = ||S-GroundTruth|| 
                loss = abs(dotproduct - label)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))

In [93]:
model = EmbModel()

(None, 1, 10)
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_id (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_emb (Embedding)             (None, 1, 10)        10          poi_id[0][0]                     
__________________________________________________________________________________________________
user_emb (Embedding)            (None, 1, 10)        10          user_id[0][0]                    
________________________________________________________________________

In [83]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)

model.compile(
    optimizer
)

#Train_data, [dataset.user_id, dataset.poi_id]. Label: ground_truth
#pgan.fit(train_data, labels, steps_per_epoch = STEPS_PER_EPOCH, epochs = EPOCHS, callbacks=[cbk])

Train the model

In [ ]:
callbacks = [keras.callbacks.EarlyStopping('val_loss', patience=10),
             keras.callbacks.ModelCheckpoint('besttest.h5', save_best_only=True)]

history = model.fit([train.movie_id, train.user_id],train.rating, batch_size=100,
                              epochs =50, validation_data = ([test.movie_id, test.user_id],test.rating),
                              verbose = 1, 
                              callbacks = callbacks)

In [ ]:
user_model.predict([np.array([10]), np.array([3])])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import matplotlib.pyplot as plt
plt.plot(history.history['loss'] , 'g')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train'], loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
print('test')

In [9]:
print('test')

test
